# Ballot-polling SPRT

This notebook explores the ballot-polling SPRT we've developed.

In [1]:
%matplotlib inline
from __future__ import division
import math
import numpy as np
import numpy.random
import scipy as sp
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sprt import ballot_polling_sprt
from hypergeometric import trihypergeometric_optim, simulate_ballot_polling_power

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


The proportion of votes for each candidate in the sample is exactly those in the population, except the population is 50 times larger. The sample of votes is made up of 2000 votes for candidate $w$, 1800 votes for candidate $\ell$, and 500 invalid votes. 

Candidate $w$ earned $46.5\%$ of the votes and candidate $\ell$ earned $41.9\%$ of the votes, corresponding to a difference of about $4.6\%$. We will test the null hypothesis that they received the same proportion of votes overall against the alternative that the reported vote totals are correct.

## Trinomial SPRT without replacement

First, suppose we don't know the number of invalid ballots. Minimize the LR over possible values.

In [2]:
alpha = 0.05
sample = [1]*2000 + [0]*1800 + [np.nan]*500
popsize = 50*len(sample)
res = ballot_polling_sprt(sample, popsize, alpha, Vw=2000*50, Vl=1800*50)
print(res)

{'upper_threshold': 20.0, 'LR': 215.50870817693178, 'lower_threshold': 0.0, 'Nu_used': 25000, 'sample_proportion': (0.46511627906976744, 0.4186046511627907, 0.11627906976744186), 'decision': 1, 'Nw_used': 95000, 'pvalue': 0.004640183723708297}


The optimization does the right thing: if we did know that there were $500 \times 50$ invalid votes in the population, we'd get the same result!

In [3]:
res = ballot_polling_sprt(sample, popsize, alpha, Vw=2000*50, Vl=1800*50, number_invalid=500*50)
print(res)

{'upper_threshold': 20.0, 'LR': 215.50870817693178, 'lower_threshold': 0.0, 'Nu_used': 25000, 'sample_proportion': (0.46511627906976744, 0.4186046511627907, 0.11627906976744186), 'decision': 1, 'Nw_used': 95000.0, 'pvalue': 0.004640183723708297}


## What happens when the reported outcome is wrong

In 100 replicates, we draw samples of 500 ballots and conduct the SPRT using the reported results as the alternative hypothesis. We never reject the null.

We do the same for samples of size 1000.

Candidate |  Reported | Actual 
---|---|---
A | 750 | 600
B | 150 | 200
Ballots | 1,000 | 1,000 
Diluted margin | 60% | 40% 

In [2]:
np.random.seed(8062018)
alpha = 0.05
population = [1]*600 + [0]*200 + [np.nan]*200
popsize = len(population)
reps = 100
rejects_sprt = 0
rejects_trihyper = 0

for i in range(reps):
    sample = np.random.choice(population, replace=False, size=50)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=750, Vl=150, null_margin=500)
    if res['decision']==1:
        rejects_sprt += 1
    res2 = trihypergeometric_optim(sample, popsize, null_margin=500)
    if res2 <= alpha:
        rejects_trihyper += 1
print("Samples of size 50, SPRT rejection rate:", rejects_sprt/reps)
print("Samples of size 50, fixed n trihypergeometric rejection rate:", rejects_trihyper/reps)

rejects_sprt = 0
rejects_trihyper = 0
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=100)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=750, Vl=150, null_margin=500)
    if res['decision']==1:
        rejects_sprt += 1
    res2 = trihypergeometric_optim(sample, popsize, null_margin=500)
    if res2 <= alpha:
        rejects_hyper += 1
        
print("Samples of size 100, SPRT rejection rate:", rejects_sprt/reps)
print("Samples of size 100, fixed n trihypergeometric rejection rate:", rejects_trihyper/reps)

Samples of size 50, SPRT rejection rate: 0.0
Samples of size 50, fixed n trihypergeometric rejection rate: 0.01
Samples of size 100, SPRT rejection rate: 0.0
Samples of size 100, fixed n trihypergeometric rejection rate: 0.0


# Another example where the reported results are wrong and consistent with the null.
The null hypothesis is that $N_w - N_\ell \leq 5$: this is true.
The alternative is that the reported results are correct: $V_w = 80$ and $V_\ell = 70$.

Candidate |  Reported | Actual 
---|---|---
A | 80 | 80
B | 70 | 75
Ballots | 165 | 165 
Diluted margin | 6% | 3% 

In [3]:
np.random.seed(8062018)
alpha = 0.05
population = [1]*80 + [0]*70 + [np.nan]*15
popsize = len(population)
reps = 100
rejects_sprt = 0
rejects_trihyper = 0
rejects_trihyper_red = 0

for i in range(reps):

    sample = np.random.choice(population, replace=False, size=100)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=80, Vl=70, null_margin=5)
    if res['decision']==1:
        rejects_sprt += 1
    res2 = trihypergeometric_optim(sample, popsize, null_margin=5)
    if res2 <= alpha:
        rejects_trihyper += 1
    if res2 <= alpha/2:
        rejects_trihyper_red += 1

print("n=1000, SPRT rejection rate:", rejects_sprt/reps)
print("n=1000, fixed n trihypergeometric rejection rate:", rejects_trihyper/reps)
print("n=1000, fixed n trihypergeometric rejection rate with alpha/2:", rejects_trihyper_red/reps)

n=1000, SPRT rejection rate: 0.0
n=1000, fixed n trihypergeometric rejection rate: 0.09
n=1000, fixed n trihypergeometric rejection rate with alpha/2: 0.06


# The reported results are wrong and inconsistent with the null.

The null hypothesis is that $N_w - N_\ell \leq 200$: this is false.
The alternative is that the reported results are correct: $V_w = 8,500$ and $V_\ell = 7,000$.
The truth is somewhere in the middle, with $N_w - N_\ell = 1,000$.

Power is not great. $n=800$ is nearly half the population.

Candidate |  Reported | Actual 
---|---|---
A | 8,500 | 8,000
B | 7,000 | 7,000
Ballots | 16,500 | 16,500 
Diluted margin | 9% | 6% 

In [6]:
np.random.seed(8062018)
alpha = 0.05
population = [1]*8000 + [0]*7000 + [np.nan]*1500
popsize = len(population)
reps = 100

rejects_sprt = 0
sprt_pvalues = []
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=1000)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=8500, Vl=7000, null_margin=200)
    if res['decision']==1:
        rejects_sprt += 1
    sprt_pvalues.append(res['pvalue'])
print("n=1000, SPRT rejection rate:", rejects_sprt/reps)
print("n=1000, median p-value:", np.median(sprt_pvalues))
      
rejects_sprt = 0
sprt_pvalues = []
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=2000)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=8500, Vl=7000, null_margin=200)
    if res['decision']==1:
        rejects_sprt += 1
    sprt_pvalues.append(res['pvalue'])
print("n=2000, SPRT rejection rate:", rejects_sprt/reps)
print("n=2000, median p-value:", np.median(sprt_pvalues))

rejects_sprt = 0
sprt_pvalues = []
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=3000)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=8500, Vl=7000, null_margin=200)
    if res['decision']==1:
        rejects_sprt += 1
    sprt_pvalues.append(res['pvalue'])
print("n=3000, SPRT rejection rate:", rejects_sprt/reps)
print("n=3000, median p-value:", np.median(sprt_pvalues))

n=1000, SPRT rejection rate: 0.02
n=1000, median p-value: 1.0
n=2000, SPRT rejection rate: 0.0
n=2000, median p-value: 1.0
n=3000, SPRT rejection rate: 0.0
n=3000, median p-value: 1.0


# The reported results are correct and inconsistent with the null.

The null hypothesis is that $N_w - N_\ell \leq 200$: this is false.
The alternative is that the reported results are correct: $V_w = 8,500$ and $V_\ell = 7,000$.
Power is improved.

Candidate |  Reported | Actual 
---|---|---
A | 8,500 | 8,500
B | 7,000 | 7,000
Ballots | 16,500 | 16,500 
Diluted margin | 9% | 6% 

In [8]:
np.random.seed(8062018)
alpha = 0.05
population = [1]*8500 + [0]*7000 + [np.nan]*1000
popsize = len(population)
reps = 100

rejects_sprt = 0
sprt_pvalues = []
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=500)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=8500, Vl=7000, null_margin=200)
    if res['decision']==1:
        rejects_sprt += 1
    sprt_pvalues.append(res['pvalue'])
print("n=500, SPRT rejection rate:", rejects_sprt/reps)
print("n=500, median p-value:", np.median(sprt_pvalues))

rejects_sprt = 0
sprt_pvalues = []
for i in range(reps):
    sample = np.random.choice(population, replace=False, size=1000)
    res = ballot_polling_sprt(sample, popsize, alpha, Vw=8500, Vl=7000, null_margin=200)
    if res['decision']==1:
        rejects_sprt += 1
    sprt_pvalues.append(res['pvalue'])
print("n=1000, SPRT rejection rate:", rejects_sprt/reps)
print("n=1000, median p-value:", np.median(sprt_pvalues))

n=500, SPRT rejection rate: 0.21
n=500, median p-value: 0.2468664086327809
n=1000, SPRT rejection rate: 0.59
n=1000, median p-value: 0.031043896965993405
